In [19]:
__author__ = 'Vladimir Iglovikov'

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score, learning_curve
from sklearn.metrics import mean_absolute_error, make_scorer
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
import pickle
import hyperopt.pyll.stochastic
import time

In [20]:
%ls

Santhosh SharmaExploratory study on ML algorithms?.ipynb  params2.txt
Vladimir Iglovikovxgb 1114?.py                            sub_v.csv
allstate_capstone-master/                                 submission.csv
handtune.ipynb                                            test.csv
hyperopt.ipynb                                            train.csv
params.txt


In [21]:
%cat params2.txt

socre:1143.8243166, params:{'reg_alpha': 16.769964407548855, 'colsample_bytree': 0.5349474195365941, 'scale_pos_weight': 3.8594086177000153, 'learning_rate': 0.03312809141914759, 'nthread': 10, 'min_child_weight': 3, 'subsample': 0.9471583516376643, 'seed': 0, 'max_depth': 6, 'gamma': 0.37515714143957835}


In [22]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test['loss'] = np.nan
joined = pd.concat([train, test])

In [23]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

In [24]:
for column in list(train.select_dtypes(include=['object']).columns):
    if train[column].nunique() != test[column].nunique():
        set_train = set(train[column].unique())
        set_test = set(test[column].unique())
        remove_train = set_train - set_test
        remove_test = set_test - set_train

        remove = (remove_train|remove_test)
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x

        joined[column] = joined[column].apply(lambda x: np.nan if x in remove else x, 1)

    joined[column] = pd.factorize(joined[column].values, sort=True)[0]

train = joined[joined['loss'].notnull()]
test = joined[joined['loss'].isnull()]

shift = 700
y = np.log(train['loss'] + shift)
ids = test['id']
X = train.drop(['loss', 'id'], 1)
X_test = test.drop(['loss', 'id'], 1)
xgtrain = xgb.DMatrix(X, label=y)
xgtest = xgb.DMatrix(X_test)

In [ ]:
time0= time.time()
space4xgb = {
    'max_depth': hp.choice('max_depth', range(3,20)),
    'min_child_weight':hp.choice('min_child_weight',range(1,10)),
    
    'learning_rate': hp.loguniform('learning_rate', -3*np.log(10), -1*np.log(10)),
    
    'scale_pos_weight': hp.uniform('scale_pos_weight', 0,10), 
    'gamma': hp.uniform('gamma',0,0.5),
    
    
    'subsample': hp.uniform('subsample',0.1,1),
    'colsample_bytree': hp.uniform('colsample_bytree',0.1,1),
    'reg_alpha':hp.uniform('reg_alpha',0,1000),
    
    'nthread': hp.choice('nthread', [10]),
    'seed':hp.choice('seed', [0])
}
minacc=1160
#{'reg_alpha': 0.8673861702351379, 'colsample_bytree': 0.8759854865376723, 'scale_pos_weight': 1, 'learning_rate': 0.06925571303753118, 'nthread': 20, 'min_child_weight': 1, 'subsample': 0.629375884543008, 'max_depth': 7, 'gamma': 0.19254932932610697}
def optf(params): 
    params['nthread']=10
    #clf = xgb.XGBRegressor(**params)
    #acc = cross_val_score(clf, X, y,cv=cv,n_jobs=1,scoring=evalerror).mean()
    bst = xgb.cv(params, xgtrain, num_boost_round=1000, nfold=5, seed=0, 
                    feval=evalerror,early_stopping_rounds=40)
    acc = bst.iloc[-1,:]['test-mae-mean']
    #print time.time()-time0
    print acc
    global minacc
    if acc < minacc:
        minacc = acc
        print 'new best:', minacc, params
        
        with open("params2.txt", "a") as text_file:
            text_file.write('socre:{}, params:{}\n'.format(minacc,params))
    return {'loss': -acc, 'status': STATUS_OK}

In [ ]:
def run_trials():

    trials_step = 1  # how many additional trials to do after loading saved trials. 1 = save after iteration
    max_trials = 1  # initial max_trials. put something small to not have to wait

    
    try:  # try to load an already saved trials object, and increase the max
        trials = pickle.load(open("/home/phe002/shicheng/dataset/hyperoptModel", "rb"))
        max_trials = len(trials.trials) + trials_step
        print("Rerunning from {} trials to {} (+{}) trials".format(len(trials.trials), max_trials, trials_step))
    except:  # create a new trials object and start searching
        trials = Trials()

    best = fmin(fn=optf, space=space4xgb, algo=tpe.suggest, max_evals=max_trials, trials=trials)
    # save the trials object
    with open("/home/phe002/shicheng/dataset/hyperoptModel", "wb") as f:
        pickle.dump(trials, f)

# loop indefinitely and stop whenever you like
while True:
    run_trials()

Rerunning from 174 trials to 175 (+1) trials
1158.9513672
new best: 1158.9513672 {'reg_alpha': 51.17342991232758, 'colsample_bytree': 0.8746115066199959, 'scale_pos_weight': 2.687076986237154, 'learning_rate': 0.014590391007893385, 'nthread': 10, 'min_child_weight': 4, 'subsample': 0.8228501992826736, 'seed': 0, 'max_depth': 10, 'gamma': 0.07453877132173166}
Rerunning from 175 trials to 176 (+1) trials
1271.6265624
Rerunning from 176 trials to 177 (+1) trials
1559.0780274
Rerunning from 177 trials to 178 (+1) trials
1882.4093752
Rerunning from 178 trials to 179 (+1) trials
3541.7228518
Rerunning from 179 trials to 180 (+1) trials
3403.0651856
Rerunning from 180 trials to 181 (+1) trials
3048.5229492
Rerunning from 181 trials to 182 (+1) trials
1424.4821288
Rerunning from 182 trials to 183 (+1) trials
3477.7983888
Rerunning from 183 trials to 184 (+1) trials
3339.2915526
Rerunning from 184 trials to 185 (+1) trials
2554.9480956
Rerunning from 185 trials to 186 (+1) trials
1217.6481692
R

In [66]:
#1148.9639894 {'reg_alpha': 0.13614510960026047, 'colsample_bytree': 0.48613283826428166, 'scale_pos_weight': 1, 'learning_rate': 0.018415349849039475, 'nthread': 10, 'min_child_weight': 3, 'subsample': 0.8045758877474857, 'max_depth': 16, 'gamma': 3.738381824865164}
'''RANDOM_STATE = 2016
params = {
    'min_child_weight': 1,
    'eta': 0.01,
    'colsample_bytree': 0.5,
    'max_depth': 12,
    'subsample': 0.8,
    'alpha': 1,
    'gamma': 1,
    'silent': 1,
    'verbose_eval': True,
    'seed': RANDOM_STATE,
    'nthread':10
}

xgtrain = xgb.DMatrix(X, label=y)
xgtest = xgb.DMatrix(X_test)

model = xgb.train(params, xgtrain, int(2012 / 0.9), feval=evalerror)'''

In [ ]:
#model.save_model('1114.model')

In [ ]:
prediction = np.exp(model.predict(xgtest)) - shift
submission = pd.DataFrame()
submission['loss'] = prediction
submission['id'] = ids
submission.to_csv('sub_v.csv', index=False)

In [ ]:
if __name__ == '__main__':
    print "a+b"